In [69]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import as_strided

In [91]:
x = np.random.rand(8000)
fs = 400
nperseg = 400
noverlap = 0
confidence = 95

In [92]:
if noverlap is None:
    noverlap = nperseg // 2
elif noverlap >= nperseg:
    raise ValueError('noverlap must be less than nperseg.')

In [93]:
N_pts = len(x)
dt = 1 / fs            
T = N_pts * dt - dt             
time = np.linspace(0, T, N_pts)
ent_std = np.std(x, ddof = 1) 
ent_mean = np.mean(x)

In [94]:
seg_pts = nperseg - noverlap
seg = int(np.ceil(N_pts / seg_pts))                  
seg_time = np.linspace(0,T,seg)
res = N_pts % seg_pts      
cls = np.array([data[i:i + seg_pts] for i in range(0, N_pts-res, seg_pts)])

TypeError: unhashable type: 'slice'

In [95]:
seg_std = np.std(cls, axis=1, ddof=1)

if res != 0:
    seg_res = data[N_pts - res:N_pts]
    if len(seg_res) != 1:
        seg_res_std = np.std(seg_res,ddof=1)
        seg_std = np.append(seg_std,seg_res_std)

cls_std = np.std(seg_std, ddof = 1)

boundUP = ent_std + cls_std
boundDW = ent_std - cls_std

rn = np.empty(seg)
for i in range(0, seg):
    if seg_std[i] > boundUP or seg_std[i] < boundDW:
        rn[i] = 1
    else:
        rn[i] = 0

N1 = N0 = 0
for i in range(0, seg):
    if rn[i] == 1.:
        N1 += 1
    else:
        N0 += 1

N = N1 + N0
Nr = 0

for i in range(1, seg):
    if rn[i] != rn[i-1]:
        Nr += 1

## Stationary limits 
if N == 0 or N == 1:
    raise ValueError('Error: check window length') 

run_mean = (2 * N1 * N0) / N + 1
run_var = (2 * N1 * N0 * (2 * N1 * N0 - N)) / (N**2 * (N - 1))

run = {'run': Nr, 'run_mean': run_mean, 'run_var': run_var}

coeff = [1.645, 1.96, 2.326, 2.576]
conf = [90, 95, 98, 99]
alpha = coeff[conf.index(confidence)]

lim_up = run_mean + alpha * np.sqrt(run_var)
lim_dw = run_mean - alpha * np.sqrt(run_var)

index_up = np.round(100 * lim_up / run_mean, 2)
index_dw = np.round(100 * lim_dw / run_mean, 2)

if Nr >= lim_dw and Nr <= lim_up:   
    bns = 'Stationary'
else:
    bns = 'Non-stationary' 
    
index = np.round( 100 * Nr / run_mean, 2)



fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(time, data, color = 'darkgray', zorder = 1, label = 'Signal')
ax.plot(seg_time, ent_mean + seg_std, color = 'C0', zorder = 2, label = 'Segments STD')
ax.hlines(ent_mean + ent_std, 0, T+dt, colors='C1', linestyles='solid', zorder = 3, label = 'STD')
ax.hlines(ent_mean + boundUP, 0, T+dt, colors='C3', linestyles='dashed', zorder = 4, label = 'Boundaries')
ax.hlines(ent_mean + boundDW, 0, T+dt, colors='C3', linestyles='dashed', zorder = 5)
ax.legend(loc=4)
ax.grid()
ax.set_xlim([0,T+dt])
ax.set_xlabel('Time [s]')
ax.set_ylabel('Amplitude [\]')
ax.set_title('Index: '+str(index) + '%\n' + bns)
plt.show()

NameError: name 'cls' is not defined

In [96]:
b.shape

(39, 400)

In [97]:
def windowed_view(arr, window, overlap = 0):
    arr = np.asarray(arr)
    window_step = window - overlap
    new_shape = arr.shape[:-1] + ((arr.shape[-1] - overlap) // window_step,window)
    new_strides = (arr.strides[:-1] + (window_step * arr.strides[-1],) + arr.strides[-1:])
    
    return as_strided(arr, shape=new_shape, strides=new_strides)

In [98]:
b = windowed_view(x,nperseg,noverlap)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class Idns(object):
    def __init__(self, x, fs, nsec, noverlap, confidence):
        self.x = x
        self.nsec = nsec
        self.fs = fs
        self.noverlap = noverlap
        self.confidence = confidence
        
        if self.nsec < 2/self.fs:
            raise ValueError('Error: nsec should be at least twice the inverse of sampling frequency')
            return None
        
    def calc(self):
        self.N_pts = len(self.x)
        self.dt = 1/self.fs            
        self.T = self.N_pts * self.dt - self.dt             
        self.time = np.linspace(0, self.T, self.N_pts)
        self.ent_std = np.std(self.x, ddof = 1) 
        self.ent_mean = np.mean(self.x)        
        coeff = [1.645, 1.96, 2.326, 2.576]
        conf = [90, 95, 98, 99]
        self.alpha = coeff[conf.index(self.confidence)]
        self.data = {'N_pts':self.N_pts,
                     'time':self.time,
                     'std':self.ent_std,
                     'mean':self.ent_mean,
                     'alpha':self.alpha}
        
        wdw_pts = int(np.floor(self.fs * self.nsec)) 
        seg_pts = wdw_pts - int(np.floor(wdw_pts * self.noverlap)) 
        seg = int(np.ceil(self.N_pts / seg_pts))                  
        self.seg_time = np.linspace(0,self.T,seg)
        res = self.N_pts % seg_pts      
        cls = np.array([self.data[i:i + seg_pts] for i in range(0, self.N_pts-res, seg_pts)])

        self.seg_std = np.std(cls, axis=1, ddof=1)

        if res != 0:
            seg_res = self.data[self.N_pts - res:self.N_pts]
            if len(seg_res) != 1:
                seg_res_std = np.std(seg_res,ddof=1)
                self.seg_std = np.append(self.seg_std,seg_res_std)
    
        cls_std = np.std(self.seg_std, ddof = 1)
    
        self.boundUP = self.ent_std + cls_std
        self.boundDW = self.ent_std - cls_std
        
        rn = np.empty(seg)
        for i in range(0, seg):
            if self.seg_std[i] > self.boundUP or self.seg_std[i] < self.boundDW:
                rn[i] = 1
            else:
                rn[i] = 0

        N1 = N0 = 0
        for i in range(0, seg):
            if rn[i] == 1.:
                N1 += 1
            else:
                N0 += 1
        
        N = N1 + N0
        self.Nr = 0

        for i in range(1, seg):
            if rn[i] != rn[i-1]:
                self.Nr += 1
        
        ## Stationary limits 
        if N == 0 or N == 1:
            raise ValueError('Error: check window length')
            return None 
        
        self.run_mean = (2 * N1 * N0) / N + 1
        self.run_var = (2 * N1 * N0 * (2 * N1 * N0 - N)) / (N**2 * (N - 1))
        
        self.run = {'run': self.Nr, 'run_mean': self.run_mean, 'run_var': self.run_var}
        
        self.lim_up = self.run_mean + self.alpha * np.sqrt(self.run_var)
        self.lim_dw = self.run_mean - self.alpha * np.sqrt(self.run_var)

        self.index_up = np.round(100 * self.lim_up / self.run_mean, 2)
        self.index_dw = np.round(100 * self.lim_dw / self.run_mean, 2)
        
        if self.Nr >= self.lim_dw and self.Nr <= self.lim_up:   
            self.bns = 'Stationary'
        else:
            self.bns = 'Non-stationary' 
            
        self.index = np.round( 100 * self.Nr / self.run_mean, 2)
        
    
    def get_data(self):
        return self.data
        
    def get_run(self):        
        return self.run
        
    def get_limits(self):
        return [self.index_dw, self.index_up]

    def get_bns(self):
        return self.bns
    
    def get_index(self):
        return self.index
    
    def get_plot(self):
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.plot(self.time, self.data, color = 'darkgray', zorder = 1, label = 'Signal')
        ax.plot(self.seg_time, self.ent_mean + self.seg_std, color = 'C0', zorder = 2, label = 'Segments STD')
        ax.hlines(self.ent_mean + self.ent_std, 0, self.T+self.dt, colors='C1', linestyles='solid', zorder = 3, label = 'STD')
        ax.hlines(self.ent_mean + self.boundUP, 0, self.T+self.dt, colors='C3', linestyles='dashed', zorder = 4, label = 'Boundaries')
        ax.hlines(self.ent_mean + self.boundDW, 0, self.T+self.dt, colors='C3', linestyles='dashed', zorder = 5)
        ax.legend(loc=4)
        ax.grid()
        ax.set_xlim([0,self.T+self.dt])
        ax.set_xlabel('Time [s]')
        ax.set_ylabel('Amplitude [\]')
        ax.set_title('Index: '+str(self.index) + '%\n' + self.bns)
        plt.show()
